In [0]:
#Opciones CSV
file_type = "csv"
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","
file_location = "/FileStore/tables/happiness_2019.csv"

# Lectura de datos usando Spark!
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

In [0]:
display(df.head(10)) #Muestra el contenido de los primeros 10 registros de la tabla happiness_2019.csv

overall rank,country or region,score,gdp per capita,social support,healthy life expectancy,freedom to make life choices,generosity,perceptions of corruption
1,Finland,7.769,1.34,1.587,0.986,0.596,0.153,0.393
2,Denmark,7.6,1.383,1.573,0.996,0.592,0.252,0.41
3,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341
4,Iceland,7.494,1.38,1.624,1.026,0.591,0.354,0.118
5,Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298
6,Switzerland,7.48,1.452,1.526,1.052,0.572,0.263,0.343
7,Sweden,7.343,1.387,1.487,1.009,0.574,0.267,0.373
8,New Zealand,7.307,1.303,1.557,1.026,0.585,0.33,0.38
9,Canada,7.278,1.365,1.505,1.039,0.584,0.285,0.308
10,Austria,7.246,1.376,1.475,1.016,0.532,0.244,0.226


In [0]:
display(dbutils.fs.ls("/FileStore/tables/happiness_2019.csv")) #dbutils.fs provides utilities for working with FileSystems. 

path,name,size
dbfs:/FileStore/tables/happiness_2019.csv,happiness_2019.csv,8510


In [0]:
#Cargamos el archivo hacia un Resilient Distributed Dataset(RDD) 
data_file = "/FileStore/tables/happiness_2019.csv"
raw_rdd = sc.textFile(data_file).cache()
raw_rdd.take(5) #show the top 5 lines of the file

Out[3]: ['overall rank,country or region,score,gdp per capita,social support,healthy life expectancy,freedom to make life choices,generosity,perceptions of corruption',
 '1,Finland,7.769,1.34,1.587,0.986,0.596,0.153,0.393',
 '2,Denmark,7.6,1.383,1.573,0.996,0.592,0.252,0.41',
 '3,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341',
 '4,Iceland,7.494,1.38,1.624,1.026,0.591,0.354,0.118']

In [0]:
type(raw_rdd)#se imprime el tipo de clase del argumento pasado como parámetro

Out[6]: pyspark.rdd.RDD

In [0]:
#En RDD, es necesario separar cada una de las entradas, antes de parsear y construir un dataframe.
csv_rdd = raw_rdd.map(lambda row: row.split(",")) 
print(csv_rdd.take(3))#print 3 rows

[['overall rank', 'country or region', 'score', 'gdp per capita', 'social support', 'healthy life expectancy', 'freedom to make life choices', 'generosity', 'perceptions of corruption'], ['1', 'Finland', '7.769', '1.34', '1.587', '0.986', '0.596', '0.153', '0.393'], ['2', 'Denmark', '7.6', '1.383', '1.573', '0.996', '0.592', '0.252', '0.41']]

In [0]:
print(type(csv_rdd))#print types

<class 'pyspark.rdd.PipelinedRDD'>

In [0]:
print('No. de columnas: ', len(csv_rdd.take(1)[0]))

No. de columnas: 9

In [0]:
#Creamos la tabla parsed_rdd.
#Vamos a importar la clase Row de la librería de pyspark.sql

from pyspark.sql import Row #Importamos la clase Row 

parsed_rdd = csv_rdd.map(lambda r: Row(
    happiness_rank = r[0],
    country = r[1],
    happiness_score = r[2],
    gdp_per_capita = r[3],
    social_support = r[4],
    health = r[5],
    freedom = r[6],
    generosity = r[7],
    corruption = r[8]
    )
)
parsed_rdd.take(5)

Out[7]: [Row(corruption='perceptions of corruption', country='country or region', freedom='freedom to make life choices', gdp_per_capita='gdp per capita', generosity='generosity', happiness_rank='overall rank', happiness_score='score', health='healthy life expectancy', social_support='social support'),
 Row(corruption='0.393', country='Finland', freedom='0.596', gdp_per_capita='1.34', generosity='0.153', happiness_rank='1', happiness_score='7.769', health='0.986', social_support='1.587'),
 Row(corruption='0.41', country='Denmark', freedom='0.592', gdp_per_capita='1.383', generosity='0.252', happiness_rank='2', happiness_score='7.6', health='0.996', social_support='1.573'),
 Row(corruption='0.341', country='Norway', freedom='0.603', gdp_per_capita='1.488', generosity='0.271', happiness_rank='3', happiness_score='7.554', health='1.028', social_support='1.582'),
 Row(corruption='0.118', country='Iceland', freedom='0.591', gdp_per_capita='1.38', generosity='0.354', happiness_rank='4', happiness_score='7.494', health='1.026', social_support='1.624')]

In [0]:
#Creamos una tabla
temp_table_name = "happiness_2019_csv"
df.createOrReplaceTempView(temp_table_name)

In [0]:
#Construimos un datraframe
df = sqlContext.createDataFrame(parsed_rdd)
display(df)

corruption,country,freedom,gdp_per_capita,generosity,happiness_rank,happiness_score,health,social_support
perceptions of corruption,country or region,freedom to make life choices,gdp per capita,generosity,overall rank,score,healthy life expectancy,social support
0.393,Finland,0.596,1.34,0.153,1,7.769,0.986,1.587
0.41,Denmark,0.592,1.383,0.252,2,7.6,0.996,1.573
0.341,Norway,0.603,1.488,0.271,3,7.554,1.028,1.582
0.118,Iceland,0.591,1.38,0.354,4,7.494,1.026,1.624
0.298,Netherlands,0.557,1.396,0.322,5,7.488,0.999,1.522
0.343,Switzerland,0.572,1.452,0.263,6,7.48,1.052,1.526
0.373,Sweden,0.574,1.387,0.267,7,7.343,1.009,1.487
0.38,New Zealand,0.585,1.303,0.33,8,7.307,1.026,1.557
0.308,Canada,0.584,1.365,0.285,9,7.278,1.039,1.505


In [0]:
#Revisamos el esquema del datraframe creado
df.printSchema()

root
-- corruption: string (nullable = true)
-- country: string (nullable = true)
-- freedom: string (nullable = true)
-- gdp_per_capita: string (nullable = true)
-- generosity: string (nullable = true)
-- happiness_rank: string (nullable = true)
-- happiness_score: string (nullable = true)
-- health: string (nullable = true)
-- social_support: string (nullable = true)

In [0]:
#Construimos una tabla temporal para correr comandos de SQL
#La tabla solo estará activa para esta sesión
df.registerTempTable("happiness")

In [0]:
#Primera consulta: agrupamos los registros de la tabla por el campo de happiness_score (usando la sintáxis del DataFrame)
display(df.groupBy('happiness_score')
          .count()
          .orderBy('count', ascending=False)
       )

happiness_score,count
5.208,2
5.529,1
5.274,1
5.175,1
2.853,1
4.534,1
7.48,1
3.663,1
5.94,1
6.321,1


In [0]:
#Misma consulta, ahora usando la sintáxis de SQL
happ_query = sqlContext.sql("""
                        SELECT happiness_score, count(*) as freq
                        FROM happiness
                        GROUP BY happiness_score
                        ORDER BY 2 DESC
                        """)
display(happ_query)

happiness_score,freq
5.208,2
5.529,1
5.274,1
5.175,1
2.853,1
4.534,1
6.321,1
7.48,1
3.663,1
5.94,1


In [0]:
#Vamos a listar qué paises son los que tienen el mismo nivel de felicidad
happ_query = sqlContext("""
                        SELECT country, happiness_score
                        FROM happiness
                        where happiness_score ==5.208
                        """)
display(happ_query)

country,happiness_score
Morocco,5.208
Azerbaijan,5.208


In [0]:
#Segunda consulta
happ_stats = sqlContext.sql("""
                            SELECT country, happiness_rank, corruption
                            FROM happiness
                            WHERE happiness_rank > 20
                            """)
display(happ_stats)

country,happiness_rank,corruption
United Arab Emirates,21,0.182
Malta,22,0.151
Mexico,23,0.073
France,24,0.183
Taiwan,25,0.097
Chile,26,0.056
Guatemala,27,0.078
Saudi Arabia,28,0.132
Qatar,29,0.167
Spain,30,0.079


In [0]:
#Guardamos la tabla de manera permanente para su posterior uso
permanent_table_name = "2019_csv"
df.write.format("parquet").saveAsTable(permanent_table_name)